In [11]:
from docplex.mp.model import Model
import pandas as pd
import numpy as np
np.seterr(all="ignore")
import math
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

def euclid_dist (x1, y1, x2, y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

#input data from .csv file
df = pd.read_csv("./coordinates.csv")
x_coord=df['x'].values
y_coord=df['y'].values
x_areas=df['x_areas'].values
y_areas=df['y_areas'].values
product_data=pd.read_csv("./product_data.csv", index_col=0)
Q=product_data['TOT_MP'].values
W=product_data['VOL_DEPLETING'].values
#MP=product_data['MP'].values 
VOL = product_data['VOL'].values
LC = product_data['#LC'].values
# dis = []
# for i in range(len(wh_coords)):
#     dis.append(euclid_dist(300,15,x_coord[i],y_coord[i]))

#input data from context
n=len(x_coord)
N=[i for i in range(1,n+1)]
V=[0,N,df.point]
q=dict(zip(df.node,df.slots))
A=[(i,j) for i in V for j in V if i!=j]
#arcs distance calculation
c={(i,j): np.hypot(cx[i]-cx[j],cy[i]-cy[j]) for i in V for j in V if i!=j}
# Zone_1=range(1,12)
# Zone_2=range(13,49)
# Zone_3=range(50,172)
# Zone_4=range(173,208)
# Zone_5=range(209,347)
# Zone_6=range(348,359)
# Zone_7=range(360,400)
# Zone_8=range(401,679)
# Zone_9=range(680,777)
# Zone_10=range(778,827)
# Stock_Zones=[Zone_1, Zone_2, Zone_3, Zone_4, Zone_5, Zone_6, Zone_7, Zone_8, Zone_9, Zone_10]

#Apc Matrix
A=np.zeros((P,Class),int) 

product_data['Class']='C' 
for i in range(len(product_data)):
    if W[i]>=15: 
        product_data['Class'][i] ='A' 
        A[[i],0] = 1 
    elif W[i]>=10 and W[i]<15:
        product_data['Class'][i]='B'
        A[[i],1]=1 
    else:
        product_data['Class'][i]='C'
        A[[i],2]=1 

restricted = []
for i in range(len(product_data)):
    if product_data['Class'][i] =='A': 
        restricted.append(i)

mdl = Model(name= 'WH_REVAMP') 

# Decision Variables
x = mdl.binary_var_matrix(L, P, name='X')  # 1 if product p is assigned to slot l, 0 otherwise
y = mdl.integer_var_matrix(L, P, name='Y')  # Number of LC of product p in slot l
s = mdl.binary_var_list(L, name='S')  # 1 if slot l is full

#CONSTRAINTS:

for l in range(L):
    # s[l] is 1 if the slot is full
    mdl.add_constraint(
        mdl.sum(VOL[p] * y[l, p] for p in range(P)) >= C * s[l]
    )
    # s[l] is 0 if the slot is not full
    mdl.add_constraint(
        mdl.sum(VOL[p] * y[l, p] for p in range(P)) <= C * s[l] + (C - 1) * (1 - s[l]) 
    ) 

# 1. At most one product type per location
for l in range(L):
    mdl.add_constraint(
        mdl.sum(x[l,p] for p in range(P)) <= 1
    )

# 2. Slot capacity
for p in range(P):
    for l in range(L):
        mdl.add_constraint(
            mdl.sum(VOL[p] * y[l,p]) <= C * x[l,p]
        )

# 3. Total LCs allocation
for p in range(P):
    mdl.add_constraint(
        mdl.sum(y[l,p] for l in range(L)) == LC[p]
    )

# 4. Consecutive slots constraint
for p in range(P):
    for l in range(L):
        while l in range(L):
            mdl.add_constraint(
                x[l+1,p]<=x[l,p]
            )


# OBJ FUNCTION
mdl.minimize(
    mdl.sum(
        mdl.sum(c[(i,j)]*x[(i,j)]*(3*A[p, 0] + 2*A[p, 1] + 1*A[p, 2]) for p in range(P) for i,j in A) 
    for l in range(L)
    )
)

#print the implementation and solution of the objective function and the constraints
print(mdl.export_to_string())

# Set CPLEX parameters for detailed logging
mdl.parameters.mip.display = 2  # Display every node and integer solution
mdl.parameters.parallel = -1 # opportunistic
mdl.parameters.workmem = 16000 # 16 GB 

# warmstart=mdl.new_solution()
# warmstart.add_var_value(x,1)
# mdl.add_mip_start(warmstart)
sol=mdl.solve(log_output=True, clean_before_solve=True)
sol.display()

#save the solution in a list
x_list = []
for l in range(L):
    tmp = False
    for p in range(P):
        if sol.get_value(x[l, p]) == 1:
            tmp = True 
            x_list.append([l+1, p+1])
    if not tmp:
        x_list.append([l+1, -1])

print(x_list)


#save data frame product_data into a .csv
product_data.to_csv("./products_data(output).csv", index=True, header=True)

# Visualization
fig = go.Figure()

# Add empty slots first
empty_slots = [l for l, p in x_list if p == -1]
empty_x = x_coord[np.array(empty_slots)-1]
empty_y = y_coord[np.array(empty_slots)-1]
empty_text = [f"Slot: {l}<br>Empty" for l in empty_slots]  # Corrected

fig.add_trace(go.Scatter(
    x=empty_x,
    y=empty_y,
    mode='markers',
    marker=dict(color='green', size=10),
    text=empty_text,
    hoverinfo='text',
    name='Vuoto'
))

# Add slots for each product class with corresponding colors
for product_class, color in zip(['A', 'B', 'C'], ['blue', 'yellow', 'purple']):
    occupied_slots = [l for l, p in x_list if p != -1 and product_data['Class'][p-1] == product_class]
    occupied_x = x_coord[np.array(occupied_slots)-1]
    occupied_y = y_coord[np.array(occupied_slots)-1]
    
    # Include product type index in hover text
    occupied_text = [
        f"Slot: {l}<br>Class: {product_data['Class'][p-1]}<br>#LC: {int(sol.get_value(y[l-1, p-1]))}<br>Product Index: {p}" 
        for l, p in x_list if p != -1 and product_data['Class'][p-1] == product_class
    ]

    fig.add_trace(go.Scatter(
        x=occupied_x,
        y=occupied_y,
        mode='markers',
        marker=dict(color=color, size=10),
        text=occupied_text,
        hoverinfo='text',
        name=f'Classe {product_class}'
    ))
# Update layout
fig.update_layout(
    title='MCV WH OPTIMIZATION',
    xaxis_title='X',
    yaxis_title='Y',
    hovermode='closest',
    showlegend=True
)

fig.show()

KeyboardInterrupt: 